In [2]:
import pandas as pd

In [18]:
columns = ["time", "sensor_id", "current_power"]
path = "15-2-27-wemo.csv"
dateparse = lambda x: pd.datetime.fromtimestamp(float(x))
data = pd.read_csv(path, names=columns, parse_dates=[0], date_parser=dateparse)

In [19]:
data.head()

,time,sensor_id,current_power
0,2015-02-27 15:03:12.240000,WeMo5,0
1,2015-02-27 15:03:12.269999,WeMo4,25640
2,2015-02-27 15:03:12.339999,WeMo6,360
3,2015-02-27 15:03:12.470000,WeMo1,159335
4,2015-02-27 15:03:12.579999,WeMo3,0


In [21]:
nonzero_data = data[data.current_power != 0]